In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

sys.path.append(str(Path.cwd().parent))

from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

ModuleNotFoundError: No module named 'torch'

In [129]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import open3d as o3d

class PointCloudDataset(Dataset):
    def __init__(self,base_dir, point_cloud_size = 5000, split = 'train', object_classes = None):
        
        self.point_cloud_size = point_cloud_size
        self.point_clouds = None
        self.split = split
        self.base_dir = base_dir
        self.object_classes = object_classes

        self.files = self.get_file_paths()
        self.point_clouds = self.get_uniform_point_clouds()


    def __len__(self):
        return len(self.point_clouds)
    
    def __getitem__(self, idx):
        return {
            "points": self.point_clouds[idx],
            "filename" : self.files[idx]
        }
    
    def get_file_paths(self):
        '''
        Return list of all filepaths in ModelNet40 that are part of split (train or test)
        If self.object_classes is populated with class names, then only files in those classes will be returned
        '''
        file_paths = []
        for root, _, files in os.walk(self.base_dir):
            for file in files:
                if file.endswith('.off'):
                    full_path = os.path.join(root, file)
                    if f'{self.split}' in full_path:
                        if self.object_classes is not None:
                            if file.split('_')[0] in self.object_classes:
                                file_paths.append(full_path)
                        else: 
                            file_paths.append(full_path)
        return file_paths
    
    
    def get_uniform_point_clouds(self):
        '''
        Return a tensor that is all point clouds of fixed size
        '''
        
        point_clouds_list = []
        for file in self.files: 
            mesh = o3d.io.read_triangle_mesh(file)
            try: 
                sampled_point_cloud = mesh.sample_points_uniformly(number_of_points = self.point_cloud_size)
                cloud = torch.tensor(np.asanyarray(sampled_point_cloud.points) ,dtype = torch.float32)
                point_clouds_list.append(F.normalize(cloud, dim = 0))
            except RuntimeError: # Some .OFF files are damaged, run repair script
                print(f'Damaged file: {file}')

        return point_clouds_list
        

In [1]:
# train_dataset_full = PointCloudDataset("../ModelNet40", 5000, 'train')
# train_loader = DataLoader(train_dataset_full, batch_size = 32, shuffle = False)
train_dataset = PointCloudDataset("../ModelNet40", 5000, 'train', object_classes= ['airplane'])
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = False)

NameError: name 'PointCloudDataset' is not defined

In [139]:
print(len(train_dataset))

645


In [154]:

pc_array = np.load("../chair_set.npy")
print(pc_array.shape)


(3746, 1024, 3)


In [3]:
class MLPEncoder(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.fc1 = nn.Linear(config.input_dim, config.hidden_dim1)
        self.fc2 = nn.Linear(config.hidden_dim1, config.hidden_dim2)
        self.fc3 = nn.Linear(config.hidden_dim2, config.latent_dim)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = self.fc3(x)

        return x

class MLPDecoder(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.fc1 = nn.Linear(config.latent_dim, config.hidden_dim2)
        self.fc2 = nn.Linear(config.hidden_dim2, config.hidden_dim1)
        self.fc3 = nn.Linear(config.hidden_dim1, config.input_dim)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = self.fc3(x)

        return x


class Autoencoder(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.encoder = MLPEncoder(config)
        self.decoder = MLPDecoder(config)

    def forward(self,x):
        latent_rep = self.encoder(x)
        out = self.decoder(latent_rep)
        return out


# class Autoencoder(nn.Module):

#     def __init__(self, config):
#         super().__init__()

#         self.fc = nn.Linear(config.input_dim, config.input_dim)

#     def forward(self,x):
#         y = self.fc(x)
#         return x + (.00001 * y)
    

class PointCloudAutoencoder(nn.Module):
    def __init__(self, config):
        super(PointCloudAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(config.input_dim, config.hidden_dim),
            nn.ReLU(),
            nn.Linear(config.hidden_dim, config.latent_dim)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(config.latent_dim, config.hidden_dim),
            nn.ReLU(),
            nn.Linear(config.hidden_dim, config.input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed


In [167]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        
        self.dec1 = nn.Linear(self.latent_size,256)
        self.dec2 = nn.Linear(256,256)
        self.dec3 = nn.Linear(256,self.point_size*3)

    def encoder(self, x): 
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
# class PointCloudAE(nn.Module):
#     def __init__(self, point_size, latent_size):
#         super(PointCloudAE, self).__init__()
        
#         self.latent_size = latent_size
#         self.point_size = point_size
        
#         self.conv1 = torch.nn.Conv1d(3, 1024, 1)
#         self.conv2 = torch.nn.Conv1d(1024, 768, 1)
#         self.conv3 = torch.nn.Conv1d(768, self.latent_size, 1)
#         self.bn1 = nn.BatchNorm1d(1024)
#         self.bn2 = nn.BatchNorm1d(768)
#         self.bn3 = nn.BatchNorm1d(self.latent_size)
        
#         self.dec1 = nn.Linear(self.latent_size,1024)
#         self.dec2 = nn.Linear(1024,2048)
#         self.dec3 = nn.Linear(2048,self.point_size*3)

#     def encoder(self, x): 
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.bn3(self.conv3(x))
#         x = torch.max(x, 2, keepdim=True)[0]
#         x = x.view(-1, self.latent_size)
#         return x
    
#     def decoder(self, x):
#         x = F.relu(self.dec1(x))
#         x = F.relu(self.dec2(x))
#         x = self.dec3(x)

#         return x.view(-1, 3, self.point_size)
    
#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x
    

In [141]:
x = next(iter(train_loader))['points']
cloud = pc.get_point_cloud(x[2])
pc.visualize_point_cloud(cloud)
# means0 = x[:,:,0].mean(dim = 0, keepdim = True)
# stds0 = x[:,:,0].std(dim = 0, keepdim = True)

# x[:,:,0] = (x[:,:,0] - means0) / 1

# means0 = x[:,:,1].mean(dim = 0, keepdim = True)
# stds0 = x[:,:,1].std(dim = 0, keepdim = True)

# x[:,:,1] = (x[:,:,1] - means0) / 1


# means0 = x[:,:,2].mean(dim = 0, keepdim = True)
# stds0 = x[:,:,2].std(dim = 0, keepdim = True)

# x[:,:,2] = (x[:,:,2] - means0) / 1

# # mean = x.mean(dim = 1, keepdim = True)
# # std = x.std(dim = 1, keepdim = True)

# # x = (x - mean) / std

# cloud = pc.get_point_cloud(x[2])
# pc.visualize_point_cloud(cloud)

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

class ReadDataset(Dataset):
    def __init__(self,  source):
     
        self.data = torch.from_numpy(source).float()


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def RandomSplit(datasets, train_set_percentage):
    lengths = [int(len(datasets)*train_set_percentage), len(datasets)-int(len(datasets)*train_set_percentage)]
    return random_split(datasets, lengths)

def GetDataLoaders(npArray, batch_size, train_set_percentage = 0.9, shuffle=True, num_workers=0, pin_memory=True):
    
    
    pc = ReadDataset(npArray)

    train_set, test_set = RandomSplit(pc, train_set_percentage)

    train_loader = DataLoader(train_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)
    test_loader = DataLoader(test_set, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size, pin_memory=pin_memory)
    
    return train_loader, test_loader

pc_array = np.load("../chair_set.npy")

train_loader, test_loader = GetDataLoaders(npArray=pc_array, batch_size= 64)

# cloud = pc.get_point_cloud(pc_array[0])
# pc.visualize_point_cloud(cloud)

In [163]:
model = PointCloudAE(1024,768).to(device)

In [172]:
point_cloud_size = 5000 


# @dataclass 
# class MLPAutoEncoderConfig:
#     input_dim = point_cloud_size * 3
#     hidden_dim1 = 3072
#     hidden_dim2 = 1048
#     latent_dim = 512

# @dataclass 
# class MLPAutoEncoderConfig:
#     input_dim = point_cloud_size * 3
#     hidden_dim1 = point_cloud_size * 3
#     hidden_dim2 = point_cloud_size * 3
#     latent_dim = point_cloud_size * 3

# @dataclass 
# class MLPAutoEncoderConfig:
#     input_dim = point_cloud_size * 3
#     hidden_dim = 2048
#     latent_dim = 512


# config = MLPAutoEncoderConfig()

# model = Autoencoder(config).to(device)
# model = PointCloudAutoencoder(config).to(device)
# model = PointCloudAE(5000,256).to(device)

optim = torch.optim.AdamW(model.parameters(), lr= 0.0005)

epochs = 1000


report_rate = 600


for epoch in range(epochs):

    running_loss = 0 

    batch_count = 0 

    for i, data in enumerate(train_loader):
        x = data.to(device).permute(0,2,1)
        # x = data['points'].to(device).permute(0,2,1)
        # x = F.normalize(x, dim = 2)
        
        optim.zero_grad()

        pred = model(x)
        pred = pred.permute(0,2,1)
        # print(x.shape)
        # print(pred.shape)
        # loss = F.mse_loss(pred, x)
        # print(x.shape)
        # print(pred.shape)
        loss, _ = chamfer_distance(x, pred) 

        # cloud = pc.get_point_cloud(x[1].T.to('cpu'))

        # pc.visualize_point_cloud(cloud)
        # p = pred[1].T.to('cpu').detach()

        # cloud = pc.get_point_cloud(p)
        # pc.visualize_point_cloud(cloud)

    
        loss.backward()

        optim.step()

        running_loss += loss.item()

        batch_count +=1

    if epoch % 10 == 9:
        print(f'Epoch {epoch:<3} Epoch Loss: {running_loss / batch_count}')
        

        # if i % report_rate == report_rate - 1:
        #     print(f'Batch {i:<3} Running Loss: {running_loss / report_rate}')
        #     running_loss = 0
    

Epoch 9   Epoch Loss: 43.05035206056991
Epoch 19  Epoch Loss: 42.993158088540135
Epoch 29  Epoch Loss: 42.90096081427808
Epoch 39  Epoch Loss: 42.92971391497918
Epoch 49  Epoch Loss: 42.90210435975273
Epoch 59  Epoch Loss: 42.79433642693286
Epoch 69  Epoch Loss: 42.81280323244491
Epoch 79  Epoch Loss: 42.805153684796025
Epoch 89  Epoch Loss: 42.76938010161778
Epoch 99  Epoch Loss: 42.729430000737025
Epoch 109 Epoch Loss: 42.71458614997144
Epoch 119 Epoch Loss: 42.66502387568636
Epoch 129 Epoch Loss: 42.6709646548865
Epoch 139 Epoch Loss: 42.66444728059589
Epoch 149 Epoch Loss: 42.69248393796525
Epoch 159 Epoch Loss: 42.601971752238725
Epoch 169 Epoch Loss: 42.58128601650022
Epoch 179 Epoch Loss: 42.516199507803286
Epoch 189 Epoch Loss: 42.511252133351455
Epoch 199 Epoch Loss: 42.47107833286501
Epoch 209 Epoch Loss: 42.44439481339365
Epoch 219 Epoch Loss: 42.43839825324292
Epoch 229 Epoch Loss: 42.411283241128025
Epoch 239 Epoch Loss: 42.42417533442659
Epoch 249 Epoch Loss: 42.346866247

In [176]:
x = next(iter(train_loader))[1]
x = F.normalize(x, dim = 0)

print(type(x))
cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)


with torch.no_grad():
    x = x.T.unsqueeze(0).to(device)
    rec_x = np.array(model(x)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)

<class 'torch.Tensor'>


In [ ]:
x.view(x.shape[0], -1).shape

torch.Size([16, 15000])

In [26]:
torch.save(model.state_dict(), 'human_auto_encoder')